# Gaussian Mixture Model (GMM) Definition

The Gaussian Mixture Model (GMM) is a family of distributions over real-valued vectors in $ \mathbb{R}^n $. The GMM is defined as follows:

1. **Assumption:** There exist $ K $ Gaussian distributions.

2. **Generation of a Sample:**
   - To generate a sample $ x \in \mathbb{R}^n $, we first select one of these $ K $ Gaussians according to a Categorical distribution:
   
     $$ Z \sim \text{Cat}(\alpha_1, \ldots, \alpha_K) $$
   
   - Here, $ Z \in \{1, 2, \ldots, K\} $ tells us which Gaussian to pick (i.e., if $ Z = 2 $, then we choose the 2nd Gaussian), and $ \alpha_k $ is the probability of choosing the $ k $-th Gaussian.
   
   - If $ Z = z $, we sample $ X $ from the $ z $-th Gaussian:
   
     $$ X \sim \mathcal{N}(\mu_z, \Sigma_z) $$

3. **Parameters of Each Gaussian:**
   - Each of the $ K $ Gaussian distributions has a separate set of parameters, i.e., a mean vector ($ \mu_z $) and a covariance matrix ($ \Sigma_z $).

4. **Parameters of the Model:**
   - The probabilities of picking the Gaussians $ \alpha_1, \ldots, \alpha_K $ are also parameters to the model.

   - The full set of model parameters is denoted as:
   
     $$ \Theta = \{\mu_1, \ldots, \mu_K, \Sigma_1, \ldots, \Sigma_K, \alpha_1, \ldots, \alpha_K\} $$

5. **Marginal Distribution:**
   - In contexts where GMMs are applied, such as data clustering, $ Z $ is usually not observed.

   - We are interested in the marginal distribution of $ X $, whose density function is given by:
   
     $$ p(x; \Theta) = \sum_{k=1}^K P(Z=k; \Theta) p(x \mid Z=k; \Theta) = \sum_{k=1}^K \alpha_k \phi(x; \mu_k, \Sigma_k) $$
   
     where $ \phi $ is the probability density function of the multivariate Gaussian distribution:

     $$ \phi(x; \mu, \Sigma) = \frac{1}{(2\pi)^{n/2} \sqrt{\text{det}(\Sigma)}} \exp\left[-\frac{1}{2}(x-\mu)^T \Sigma^{-1} (x-\mu)\right] $$

6. **Example:**
   - Here's an example density function for a two-dimensional GMM with three Gaussians ($ K = 3 $):

     $$ p(x; \Theta) = \alpha_1 \phi(x; \mu_1, \Sigma_1) + \alpha_2 \phi(x; \mu_2, \Sigma_2) + \alpha_3 \phi(x; \mu_3, \Sigma_3) $$

<center>

![drawing](assets/gmm.png)

</center>


<center>

![drawing](assets/gmm_animation.gif)

</center>

# A Model for Data Clustering

Imagine having a dataset of points $x_1, x_2, \ldots, x_n \in \mathbb{R}^n$, and the objective is to identify clusters within these data points, where points within a cluster are more similar to each other than to points outside their cluster. Gaussian Mixture Models (GMMs) offer a framework for discovering such clusters.

## Assumptions

To cluster the data, we assume a strong hypothesis: the data points are samples from a GMM with $K$ Gaussians, where $K$ represents the assumed number of clusters. Unfortunately, the parameters of the GMM (mean and covariance of each Gaussian) and the assignments of data points to Gaussians are unknown. In other words, we lack information about the GMM's parameters $\Theta$ and the latent variables $Z_1, Z_2, \ldots, Z_n$.

## Clustering Steps

To perform clustering on $x_1, x_2, \ldots, x_n$, we can follow these steps. First, we need to estimate the values for $\Theta$. Assuming we have an estimate denoted as

$$
\hat{\Theta} = \{\hat{\mu}_1, \ldots, \hat{\mu}_K, \hat{\Sigma}_1, \ldots, \hat{\Sigma}_K, \hat{\alpha}_1, \ldots, \hat{\alpha}_K\}
$$

Given this estimate, we can assign $x_i$ to the Gaussian (i.e., cluster) that was most likely to generate $x_i$:

$$
\text{arg max}_{k \in \{1, \ldots, K\}} P(Z_i = k | x_i; \hat{\Theta})
$$

Using Bayes’ rule, we compute

$$
P(Z_i = k | x_i; \hat{\Theta}) = \frac{\phi(x_i | \hat{\mu}^k, \hat{\Sigma}^k) \hat{\alpha}^k}{\sum_{h=1}^K \phi(x_i | \hat{\mu}^h, \hat{\Sigma}^h) \hat{\alpha}^h}
$$


## Maximum-Likelihood Estimation

Returning to the task of estimating values for $\Theta$, we approach it via maximum-likelihood:

$$
\hat{\Theta} := \text{arg max}_{\Theta} \prod_{i=1}^n p(x_i; \Theta)
$$

The optimization problem can be solved using the Expectation-Maximization (EM) algorithm, providing a straightforward approach to address the challenge of estimating the GMM parameters.

# Deriving the EM algorithm for GMM’s

## Derivation of the E-step

Here, we derive the Q-function at the *t*-th iteration. Let $X := \{x_1, \ldots, x_n\}$ be the collection of observed data (i.e., the data points) and $Z := \{z_1, \ldots, z_n\}$ be the collection of latent data (i.e., which Gaussian generated each data point). Recall, the Q-function is defined as

$$Q_t(\Theta) := \mathbb{E}_{Z \mid X, \Theta_t}[\log p(X, Z; \Theta)]$$

Deriving the Q-function entails calculating an analytical form of this expectation so that we can implement it in a computer program. For GMM’s that derivation is:

$$Q_t(\Theta) := \mathbb{E}_{Z \mid X, \Theta_t}[\log p(X, Z; \Theta)] = \sum_{i=1}^n \mathbb{E}_{z_i \mid x_i, \Theta_t}[\log p(x_i, z_i; \Theta)] \quad \text{(by the linearity of expectation)}$$

$$= \sum_{i=1}^n \sum_{k=1}^K P(z_i=k \mid x_i; \Theta_t) \log p(x_i, z_i; \Theta)$$

$$= \sum_{i=1}^n \sum_{k=1}^K \frac{ P(x_i \mid z_i=k; \Theta_t) P(z_i=k; \Theta_t)}{\sum_{h=1}^K P(x_i \mid z_i=h; \Theta_t) P(z_i=h; \Theta_t)} \log p(x_i, z_i; \Theta)$$

$$= \sum_{i=1}^n \sum_{k=1}^K \frac{\alpha_{t,k} \phi(x_i; \mu_{t,k}, \Sigma_{t,k})}{\sum_{h=1}^K \alpha_{t,h} \phi(x_i; \mu_{t,h}, \Sigma_{t,h})} \log(\alpha_k \phi(x_i; \mu_k, \Sigma_k))$$

$$= \sum_{i=1}^n \sum_{k=1}^K \gamma_{t,i,k} \log(\alpha_k \phi(x_i; \mu_k, \Sigma_k))$$

where we let

$$\gamma_{t,i,k} := \frac{\alpha_{t,k} \phi(x_i; \mu_{t,k}, \Sigma_{t,k})}{\sum_{h=1}^K \alpha_{t,h} \phi(x_i; \mu_{t,h}, \Sigma_{t,h})}$$





```python
    def _get_likelihoods(self, x: np.ndarray) -> np.ndarray:
        """
        Compute the likelihoods for each data point and Gaussian component.

        Args:
        - x (numpy.ndarray): Input data.

        Returns:
        - numpy.ndarray: Likelihood values.
        """
        likelihoods = []
        for c in range(self.n_components):
            likelihoods.append(self.pdf_multivariate(x, mu=self.means[c], sigma=self.sigmas[c]))
        return np.stack(likelihoods, axis=1)
        
    def _E_step(self, x: np.ndarray) -> None:
        """
        Perform the Expectation (E) step of the EM algorithm.

        Args:
        - x (numpy.ndarray): Input data.
        """
        likelihoods = self._get_likelihoods(x)
        weighted_likelihoods = likelihoods*self.alpha_c
        self.gamma = weighted_likelihoods/np.sum(weighted_likelihoods, axis=1)[:, np.newaxis]
```

## Derivation of the M-step

In the M-step, the objective is to find $\Theta_{t+1}$ that maximizes the Q-function. Notably, $\alpha_1, \ldots, \alpha_k$ represent the probabilities of a given point being sampled from each Gaussian. These probabilities must sum to one. Consequently, when optimizing the Q-function concerning these parameters, it is imperative to adhere to the constraint that they sum to one:

$$
\Theta_{t+1} := \arg \max_\Theta Q_t(\Theta)
$$

subject to

$$
\sum_{k=1}^K \alpha_k = 1
$$

Given the equality constraint and the continuity of the objective and constraint, solving this optimization problem involves leveraging Lagrange multipliers. The Lagrangian is formulated as follows:

$$
L(\Theta, \lambda) := \sum_{i=1}^n \sum_{k=1}^K \gamma_{t,i,k} \log(\alpha_k \phi(x_i; \mu_k, \Sigma_k)) + \lambda \left(\sum_{k=1}^K \alpha_k - 1\right)
$$

The next step is to solve for $\Theta$ by setting the derivative of the Lagrangian equal to zero.

### Solving for $\alpha_1, \ldots, \alpha_K$

For a specific $k$,

$$
\frac{\partial L(\Theta, \lambda)}{\partial \alpha_k} = \frac{1}{\alpha_k} \sum_{i=1} \gamma_{t,i,k} + \lambda
$$

Setting this expression to zero and solving for $\alpha_k$, we get

$$
0 = \frac{1}{\alpha_k} \sum_{i=1} \gamma_{t,i,k} + \lambda \\ \Rightarrow \alpha_k =  -\frac{1}{\lambda} \sum_{i=1}^n \gamma_{t,i,k}
$$

Substituting this into the constraint $\sum_{k=1}^K \alpha_k = 1$, we can solve for $\lambda$,

$$
\sum_{k=1}^K -\frac{1}{\lambda} \sum_{i=1}^n \gamma_{t,i,k} = 1 \\  \Rightarrow \lambda = -\sum_{i=1}^n \sum_{k=1}^K \gamma_{t,i,k} \\ \Rightarrow \lambda=  -n \quad \text{because } \sum_{k=1}^K \gamma_{t,i,k} = 1
$$

Finally, substituting $\lambda$ back into the equation setting the derivative of the Lagrangian to zero, we can solve for the final value of $\alpha_k$:

$$
0 = \frac{1}{\alpha_k} \sum_{i=1}^n \gamma_{t,i,k} + \lambda \\ \Rightarrow 0 = \frac{1}{\alpha_k} \sum_{i=1}^n \gamma_{t,i,k} - n \\ \Rightarrow \alpha_k = \frac{1}{n} \sum_{i=1}^n \gamma_{t,i,k}
$$

And there we have successfully solved for the $\alpha_1, \ldots, \alpha_K$ parameters that maximize the Q-function.



### Solving for $\mu_k$

Now, we’ll move on to the Gaussian means. First, we compute the derivative of the Lagrangian with respect to $\mu_k$ for some $k$:

$$\frac{\partial L(\Theta, \lambda)}{\partial \mu_k} := \sum_{i=1}^n \gamma_{t,i,k} \frac{\partial}{\partial \mu_k} \log\left(\alpha_k \phi(x_i; \mu_k, \Sigma_k)\right)$$

$$= \sum_{i=1}^n \gamma_{t,i,k} \left[-2 \Sigma_k^{-1} (x_i - \mu_k)\right

]$$

The last step of this derivation follows from the fact that

$$\frac{\partial}{\partial s} (x - s)^T W (x - s) = -2 W^{-1} (x - s)$$

as described in Equation 86 in The Matrix Cookbook.

Now, setting the derivative of the Lagrangian, with respect to the mean vector, to the zero vector, we can solve for $\mu_k$:

$$0 = \sum_{i=1}^n \gamma_{t,i,k} \left[-2 \Sigma_k^{-1} (x_i - \mu_k)\right]  \\
0 = \Sigma_k^{-1} \sum_{i=1}^n \gamma_{t,i,k} (x_i - \mu_k) \\
\Sigma_k 0 = \Sigma_k\Sigma_k^{-1} \sum_{i=1}^n \gamma_{t,i,k} (x_i - \mu_k) \\
0 = \sum_{i=1}^n \gamma_{t,i,k} (x_i - \mu_k) \\
\Rightarrow \mu_k = \frac{1}{\sum_{i=1}^n \gamma_{t,i,k}} \sum_{i=1}^n \gamma_{t,i,k} x_i$$

And there we’ve solved for the Gaussian means that maximize the Q-function.

### Solving for $\Sigma_k$

Finally, we solve for the covariance matrices that maximize the Q-function. We first compute the gradient with respect to the covariance matrix $\Sigma_k$ for some $k$:

$$\frac{\partial L(\Theta, \lambda)}{\partial \Sigma_k} := \sum_{i=1}^n \gamma_{t,i,k} \frac{\partial}{\partial \Sigma_k} \log\left(\alpha_k \phi(x_i; \mu_k, \Sigma_k)\right)$$

$$= \sum_{i=1}^n \gamma_{t,i,k} \left[-\frac{1}{2} \frac{\partial}{\partial \Sigma_k}\log \det \left(\Sigma_k \right)- \frac{1}{2}  \frac{\partial}{\partial \Sigma_k} (x_i - \mu_k)^{T} \Sigma_k^{-1} (x_i - \mu_k)\right]$$


$$= -\frac{1}{2} \left(\sum_{i=1}^n \gamma_{t,i,k} \right) \Sigma_k^{-1} - \Sigma_k^{-1} \left( \sum_{i=1}^n \gamma_{t,i,k} (x_i - \mu_k)(x_i - \mu_k)^T \right) \Sigma_k^{-1}$$


Now, setting $\frac{\partial L(\Theta, \lambda)}{\partial \Sigma_k}$ to the zero matrix in $R^{n \times n}$, we can solve for $\Sigma_k,t+1$:

$$
\begin{align*}
\boldsymbol{0} &= \frac{1}{2} \left(\sum_{i=1}^n \gamma_{t,i,k}\right)\boldsymbol{\Sigma}_k^{-1} - \boldsymbol{\Sigma}_k^{-1} \left(\sum_{i=1}^n \gamma_{t,i,k} (\boldsymbol{x}_i - \boldsymbol{\mu}_i)(\boldsymbol{x}_i - \boldsymbol{\mu}_i)^T\right) \boldsymbol{\Sigma}_k^{-1} \\
\implies \boldsymbol{0} &= \boldsymbol{\Sigma}_k^{-1} \left[\left(\sum_{i=1}^n \gamma_{t,i,k}\right)\boldsymbol{I} - \left(\sum_{i=1}^n \gamma_{t,i,k} (\boldsymbol{x}_i - \boldsymbol{\mu}_i)(\boldsymbol{x}_i - \boldsymbol{\mu}_i)^T\right) \boldsymbol{\Sigma}_k^{-1} \right] \quad \quad \boldsymbol{I} \ \text{is the identity matrix} \\
\implies \boldsymbol{0} &= \left(\sum_{i=1}^n \gamma_{t,i,k}\right)\boldsymbol{I} - \left(\sum_{i=1}^n \gamma_{t,i,k} (\boldsymbol{x}_i - \boldsymbol{\mu}_i)(\boldsymbol{x}_i - \boldsymbol{\mu}_i)^T\right) \boldsymbol{\Sigma}_k^{-1} \quad \quad \text{left multiply both sides by} \ \boldsymbol{\Sigma}_k \\
\implies \left(\sum_{i=1}^n \gamma_{t,i,k}\right)\boldsymbol{I} &= \left(\sum_{i=1}^n \gamma_{t,i,k} (\boldsymbol{x}_i - \boldsymbol{\mu}_i)(\boldsymbol{x}_i - \boldsymbol{\mu}_i)^T\right) \boldsymbol{\Sigma}_k^{-1} \\
\implies \left(\sum_{i=1}^n \gamma_{t,i,k}\right)\boldsymbol{\Sigma}_k &= \sum_{i=1}^n \gamma_{t,i,k} (\boldsymbol{x}_i - \boldsymbol{\mu}_i)(\boldsymbol{x}_i - \boldsymbol{\mu}_i)^T \\
\boldsymbol{\Sigma}_k &= \frac{1}{\sum_{i=1}^n \gamma_{t,i,k}}\sum_{i=1}^n \gamma_{t,i,k} (\boldsymbol{x}_i - \boldsymbol{\mu}_i)(\boldsymbol{x}_i - \boldsymbol{\mu}_i)^T
\end{align*}
$$

```python
    def _M_step(self, x: np.ndarray) -> None:
        """
        Perform the Maximization (M) step of the EM algorithm.

        Args:
        - x (numpy.ndarray): Input data.
        """
        gamma_summs = np.sum(self.gamma, axis=0)
        for c in range(self.n_components):
            self.means[c] = np.sum(x*self.gamma[:, c, np.newaxis], axis=0)/gamma_summs[c]
            self.sigmas[c] = ((x - self.means[c]).T @ ((x - self.means[c])*self.gamma[:, c, np.newaxis]))/gamma_summs[c]
        alpha_c = gamma_summs/x.shape[0]
        self.alpha_c = alpha_c

```

<center>

![drawing](assets/gmm_animation.gif)

</center>